# e-magyar elemzés

---

(2021. 04. 16.)

Mittelholcz Iván

## 1. Az e-magyar használata

Az elemzendő szöveg:

In [ ]:
!cat orkeny.txt

Az e-magyar legfrissebb verziójának letöltése:

In [ ]:
!docker pull mtaril/emtsv:latest

Az *orkeny.txt* elemzése, az eredmény kiírása az *orkény.tsv* fájlba:

In [ ]:
!docker run --rm -i mtaril/emtsv:latest tok,morph,pos,ner <orkeny.txt >orkeny.tsv

Magyarázatok:

- `!docker run --rm -i mtaril/emtsv:latest`: Az *e-magyar* futtatása
- `tok,morph,pos,ner`: a használt modulok felsorolása
    - `tok`: tokenizálás
    - `morph`: morfológiai elemzés
    - `pos`: szófaji egyértelműsítés
    - `ner`: névelem felismerés
- `<orkeny.txt`: Az elemzendő szöveg beolvasása az *orkeny.txt* fájlból.
- `>orkeny.tsv`: Az elemzés kiírása az *orkeny.tsv* fájlba.

## 2. Az elemzés beolvasása *pandas DataFrame*-be

In [ ]:
import pandas as pd

df = pd.read_csv('orkeny.tsv', sep='\t')
df.head(50)

Oszlopok:

- *form*: A tokenizáló modul (*tok*) kimenete. A szövegben található tokeneket (szóalakokat, írásjeleket) tartalmazza.
- *wsafter*: Szintén a tokenizáló kimenete. A tokenek után található *whitespace* karaktereket tartalmazza.
- *anas*: A morfológiai elemző kimenete (*morph*). Szögletes zárójelpáron belül tartalmazza a lehetséges morfológiai elemzések listáját. Használt morfológiai kódok leírása [itt](https://e-magyar.hu/hu/textmodules/emmorph_codelist).
- *lemma*: A szófaji egyértelműsítő kimenete (*pos*). A legvalószínűbb morfológiai elemzéshez tartozó lemmát tartalmazza. 
- *xpostag*: Szintén a szófaji egyértelműsítő kimenete (*pos*). A legvalószínűbb morfológiai elemzést tartalmazza. 
- *NER-BIO*: A tulajdonnév felismerő modul kimenete (*ner*). Leírása [itt](https://e-magyar.hu/hu/textmodules/emner).

## 3. Elemzesek


### Felhasználási esetek

#### Feladatok, amikhez egyszerre elég egy sort (*row*) figyelembe venni

- Szűrni bizonyos pos-tagekre, pl. keressük a múltidejű igéket.
- Adott lemmahalmaz múltidejű előfordulásai.
- Több morfológiai jegy figyelembevétele: pl. adott lemmahalmaz múltidejű előfordulásai egyeszám elsőszemélyben ill. többesszám elsőszemélyben.

A végén számolni kéne ezeket: az összes tokenszámhoz, vagy szószámhoz, vagy az összes igéhez képest milyen arányban fordulnak elő ezek az alakok.

#### Feladatok, amikhez több sort kell figyelembe venni

- Van-e személyes névmás az ige mellett? Pl. "éldegéltem" vs. "én éldegéltem".
- Főnévnek van-e jelzője?
- Igének van-e határozószava?
- Tagmondat szintű elemzés: keressük azon tagmondatokat, amikben van kötőszó, de nincs múltidejű igealak.

Ezeket megint arányítani kell: az összes főnévből mennyinek van jelzője, az összes igéből mennyinek van határozója.

#### Feladatok, amikhez az eredeti szöveget kell módosítani

- Potenciálisan többszavas kifejezések keresése ([emterm](https://github.com/dlt-rilmta/emterm)!).
- Szövegbe visszaírni elemzések eredményét XML-szerűen, pl. <érzelmi_kifejezés>...</érzelmi_kifejezés>

### 3.1. Egy soros feladatok

In [ ]:
# multideju igek aranya

def is_not_punct(row):
    pos = row['xpostag']
    return not pos.startswith('[Punct]')

def is_verb(row):
    pos = row['xpostag']
    return pos.startswith('[/V]')

def is_past_verb(row):
    pos = row['xpostag']
    return pos.startswith('[/V][Pst.')

mask0 = df.apply(is_not_punct, axis=1)
mask1 = df.apply(is_verb, axis=1)
mask2 = df.apply(is_past_verb, axis=1)

count_word = len(df[mask0])
count_verb = len(df[mask1])
count_past_verb = len(df[mask2])

print('multideju igek / osszes token: ', count_past_verb/len(df))
print('multideju igek / osszes szo: ', count_past_verb/count_word)
print('multideju igek / osszes ige: ', count_past_verb/count_verb)
df[mask1]

In [ ]:
# egyesszam 3. szemelyu igek

def is_3sg_verb(row):
    pos = row['xpostag']
    return pos.startswith('[/V]') and '3Sg' in pos

mask3 = df.apply(is_3sg_verb, axis=1)

count_3sg_verb = len(df[mask3])

print('3sg igek / osszes token: ', count_3sg_verb/len(df))
print('3sg igek / osszes szo: ', count_3sg_verb/count_word)
print('3sg igek / osszes ige: ', count_3sg_verb/count_verb)
df[mask3]

In [ ]:
# adott lemmahalmaz keresése

def is_lemma_in_set(row):
    lemma = row['lemma']
    lemmaset = {'iszik', 'van'}
    return lemma in lemmaset

mask4 = df.apply(is_lemma_in_set, axis=1)

count_lemmaset = len(df[mask4])

print('halmazban levo igek / osszes token: ', count_lemmaset/len(df))
print('halmazban levo igek / osszes szo: ', count_lemmaset/count_word)
print('halmazban levo igek / osszes ige: ', count_lemmaset/count_verb)

df[mask4]